<a href="https://colab.research.google.com/github/LeFalko/detect_CS/blob/master/Detect_CS_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

#download U'n'Eye
!git clone https://github.com/berenslab/uneye.git

# remove folders and files that are not needed
os.chdir('uneye')
import shutil
files = ['README.md','UnEye.ipynb','logo.jpeg','requirements_lin.txt','requirements_mac.txt','requirements_wind.txt','setup.py']
for i,f in enumerate(files):
    try:
        os.remove(f)
    except:
        print("File already deleted: ", f)
          
folders = ['.git','analysis scripts','data']
for i,f in enumerate(folders) :
    try:
        shutil.rmtree(f)
    except:
        print("Folder already deleted: ", f)
os.makedirs('training',exist_ok = True)
os.makedirs('work',exist_ok = True)
os.makedirs('work/LabelYourData',exist_ok = True)
os.makedirs('work/Output',exist_ok = True)
os.makedirs('work/TrainYourNetwork',exist_ok = True)

# add a patch to use U'n'Eye as CS detector
!git clone https://github.com/LeFalko/detect_CS.git

# to do math operations
import numpy as np
import scipy.io as io

# to read .mat files
!pip install mat4py
!pip install mat73
import mat4py
# to do read .pkl files
import pandas as pd

# to do clustering
!pip install hdbscan

import uneye
!pip install umap-learn
from detect_CS import *
#to download files from the internet
import urllib

Cloning into 'uneye'...
remote: Enumerating objects: 990, done.
remote: Total 990 (delta 0), reused 0 (delta 0), pack-reused 990
Receiving objects: 100% (990/990), 169.38 MiB | 23.58 MiB/s, done.
Resolving deltas: 100% (211/211), done.
Checking out files: 100% (489/489), done.
Cloning into 'detect_CS'...
remote: Enumerating objects: 547, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 547 (delta 122), reused 94 (delta 41), pack-reused 351
Receiving objects: 100% (547/547), 2.13 MiB | 3.36 MiB/s, done.
Resolving deltas: 100% (329/329), done.
     |████████████████████████████████| 5.2 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330729 sha256=efae7c2d730e6c3b4b26bd384548c0a06bc8d97558538bcb786f230aae0fc126
  Stored in directory: /root/.cac

In [ ]:
file_link = 'https://ndownloader.figshare.com/files/21765003?private_link=a1f3eb9510b0a628ed28'

urllib.request.urlretrieve(file_link,'PC_EMI_S4_Right_10-08-2017.mat') # Download the file from Figshare server
example_file = mat4py.loadmat('PC_EMI_S4_Right_10-08-2017.mat') # Load the file in memory

In [ ]:
print(example_file.keys())
sampling_rate = 25000; # Hz

RAW = np.array(example_file['RAW'])
HIGH = np.array(example_file['HIGH'])
Labels = np.array(example_file['Labels'])
Interval_inspected = np.array(example_file['Interval_inspected'])

dict_keys(['RAW', 'HIGH', 'Labels', 'Interval_inspected'])


In [ ]:
sampling_rate = 25000; # Hz

LFP,High,Labels,Interval_inspected = load_data(filename = 'PC_EMI_S4_Right_10-08-2017.mat',# path of the file
                          # it can be a .mat file, a .pkl file or a .csv file
                          # set to [] if you use .csv files
                            field_LFP = ['RAW'],# in a .pkl file where the LFP data is in the key ['lfp'] then set: field_LFP = 'lfp'
                            # in a .mat file where the LFP data is in the variable lfp then set: field_LFP = 'lfp'
                            # in a .mat file where the LFP data is in the structure Data.lfp then set: field_LFP = ['Data','lfp']
                            # with .csv files this is the path to the file containing LFP
                            # load data will also automatically filter the wide-band signal if it is provided instead of the LFP hence we give the ['RAW'] field
                            field_high_pass = ['HIGH'], # same as above but the data has to be already high-passed
                            field_label = ['Labels'], # same as above
                            field_intervs = ['Interval_inspected'], # same as above
                            sampling_freq = sampling_rate # sampling frequency in Hz
                            )
compLFP,compHIGH,compLabels = concatenate_segments(LFP,High,Interval_inspected,Labels)

[-0.18677687 -0.17702209 -0.16890409 ...  0.04995807  0.03908252
  0.0301529 ]


In [ ]:
# Download the training set of our study
# It is already normalized and concatenated
file_link ='https://ndownloader.figshare.com/files/21809856?private_link=31e490395584c1eea69b'
urllib.request.urlretrieve(file_link,'training_set.mat') # The file training_set.mat contains all normalized and concatenated segments from our dataset

DATA = io.loadmat('training_set.mat')

ID = DATA['ID']
bigTraining_LFP = DATA['LFP'].squeeze()
bigTraining_HIGH = DATA['HIGH'].squeeze()
bigTraining_LABEL = DATA['LABEL'].squeeze()

def squizz(data):
    data = [x.squeeze() for x in data]
    return(data)
bigTraining_LFP = squizz(bigTraining_LFP)
bigTraining_HIGH = squizz(bigTraining_HIGH)
bigTraining_LABEL = squizz(bigTraining_LABEL)
DATA = None

In [ ]:
length = int(sampling_rate*1) # we will use only 1 s of data from each cell
trainingLFP = []
trainingHIGH = []
trainingLabel = []
for ii,txt in enumerate(ID):
    if not(txt == 'PC_EMI_S4_Right_10-08-2017'):
            trainingLFP.append(bigTraining_LFP[ii][:length])
            trainingHIGH.append(bigTraining_HIGH[ii][:length])
            trainingLabel.append(bigTraining_LABEL[ii][:length])
    else:
        print('PC_EMI_S4_Right_10-08-2017 is skipped')
trainingLFP = np.concatenate(trainingLFP)  
trainingHIGH = np.concatenate(trainingHIGH)  
trainingLabel = np.concatenate(trainingLabel)  

PC_EMI_S4_Right_10-08-2017 is skipped


In [ ]:
# size of the convolutional and maxpooling operations
ks = 9 # convolution kernel size. NEED TO BE ODD.
mp = 7 # maxpooling size. NEED TO BE ODD.

# the number of bins (nb) taken into account by the network is given by
# nb = mp**2+mp**2*ks+(mp*ks)-mp+2*ks-2 

weights_name = 'Test' # the weights will be saved in the folder 'training'
sampfreq = 25000 # Sampling frequency of your signal in Hz
val_samples = 10 # If you have a small training set you should consider reducing this number.
                 # Otherwise the training will show an error
#########################
model = uneye.DNN(max_iter = 3000, ks=ks,mp=mp, weights_name=weights_name,
                  sampfreq = sampfreq,augmentation = False,val_samples = val_samples, doDiff = False)

model.train(trainingLFP,trainingHIGH,trainingLabel)

Number of classes: 2
Using GPU: True
Training. Please wait.
Early stopping at epoch 99 before overfitting occurred.
Model parameters saved to ./training/Test


# Testing

In [ ]:
# size of the convolutional and maxpooling operations
ks = 9 # convolution kernel size. NEED TO BE ODD.
mp = 7 # maxpooling size. NEED TO BE ODD.

# the number of bins (nb) taken into account by the network is given by
# nb = mp**2+mp**2*ks+(mp*ks)-mp+2*ks-2 

weights_name = 'Test' # the weights will be saved in the folder 'training'
sampfreq = 25000 # Sampling frequency of your signal in Hz
val_samples = 10 # If you have a small training set you should consider reducing this number.
                 # Otherwise the training will show an error
field_LFP = ['RAW']
field_high_pass = ['HIGH']

In [ ]:
testing_folder = '/gdrive/My Drive/uneye/work/LabelYourData/'
testing_files = os.listdir(testing_folder)
output_folder = '/gdrive/My Drive/uneye/work/Output/'
for tf in testing_files:
    LFP,High,_,_ = load_data(filename = testing_folder+tf,field_LFP = field_LFP,field_high_pass = field_high_pass)
    
    output = detect_CS(weights_name,
                   LFP,
                   High,
                   # The arguments above need to be given is this order
                
                   # The arguments bellow are optional

                   output_name = 'results.mat', # Here you write the path and name of the
                                       # file to save a .mat file of the results
                                       # default value: None , 
                   
                   sampling_frequency = sampfreq, # In Hz. Default value: 25000
                   ks=ks, # convolution kernel size. NEED TO BE ODD. Default value: 9
                   mp=mp, # maxpooling size. NEED TO BE ODD. Default value: 7
                   realign = True, # Realign waveforms in the range given by the variable 'alignment_w' before clustering
                                   # default value: True
                   alignment_w = (-.5,2), # Time (ms) before and after the putative CS onset used to realign onset
                                         # default value: (-.5,2)
                   cluster = True, # Perform dimensionality reduction and clustering in the window given by the parameter 'cluster_w'
                                  # default value: True
                   cluster_w = (0,2), # Time (ms) before and after the putative CS onset used to perform dimensionality reduction
                                      # default value: (0,2)
                   plot_w= (-4,8), # Time (ms) before and after the putative CS onset used to display waveforms
                                    # default value: (-4,8)
                   exlude_w = 3, # Minimum interval (in ms) of high predictive probability to consider a cluster of putative CSs as a real CSs
                                # default value: 3
                   plot = True, # Display waveforms (as in figure 2 of our paper)
                                # default value: False
                   plot_only_good = False, # If set to True, only the putative CSs kept by the algorithm are displayed
                                           # If set to False, the events that are too short are also displayed
                                           # default value: True
                   )

    
    

FileNotFoundError: ignored